# Michael Mascilli

In [2]:
import pandas as pd

chip = pd.read_csv("City-Of-Pittsburgh-Traffic-Count.csv")
chip.head()

,id,device_id,record_oid,count_start_date,count_end_date,average_daily_car_traffic,average_daily_bike_traffic,counter_number,counter_type,speed_limit,...,longitude,latitude,neighborhood,council_district,ward,tract,public_works_division,pli_division,police_zone,fire_zone
0,1011743669,85,1.445865e+09,2019-04-18,2019-04-26,4949.0,NaN,6,StatTrak,25.0,...,-79.967772,40.455733,Polish Hill,7.0,6.0,42003060500,6.0,6.0,2.0,2-6
1,1026101993,140,1.121444e+09,2019-01-24,NaN,NaN,NaN,NaN,Intersection Study,NaN,...,-79.952249,40.466157,Central Lawrenceville,7.0,9.0,42003090200,2.0,9.0,2.0,3-6
2,1032382575,11,1.539893e+09,2018-08-28,2018-09-04,NaN,NaN,NaN,NaN,35.0,...,-80.076469,40.460717,Windgap,2.0,28.0,42003563000,5.0,28.0,6.0,1-16
3,103627606,9,7.341951e+08,2018-07-17,2018-08-01,2741.0,NaN,NaN,StatTrak,25.0,...,-79.914335,40.437379,Squirrel Hill South,5.0,14.0,42003140800,3.0,14.0,4.0,2-18
4,1039546167,144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-80.019211,40.490794,Perry North,1.0,26.0,42003260200,1.0,26.0,1.0,1-15


In [3]:
# This will display each unique item and how many of them there are
chip['neighborhood'].value_counts()

Squirrel Hill South    28
Bloomfield             21
Shadyside              18
Highland Park          16
Mount Washington       16
                       ..
Duquesne Heights        1
Middle Hill             1
Lincoln Place           1
Fineview                1
East Allegheny          1
Name: neighborhood, Length: 74, dtype: int64

In [4]:
traffic = {}

# Create a dictionary based off the item name and the quantity of that item
for index, row in chip.iterrows():
    # We must include the second condition since the first time we add a neighborhood, it might be NaN, which we cannot use
    if row["neighborhood"] not in traffic and row["average_daily_car_traffic"] != "NaN":
        traffic[row["neighborhood"]] = row["average_daily_car_traffic"]
        
# specify the column names separately
column_names = ["frequency"]
items = pd.DataFrame(traffic, column_names)
items

,Polish Hill,Central Lawrenceville,Windgap,Squirrel Hill South,Perry North,Central Northside,Squirrel Hill North,Central Oakland,East Liberty,Greenfield,...,New Homestead,Lincoln Place,NaN,Oakwood,Banksville,Chartiers City,Summer Hill,Troy Hill,Middle Hill,Crawford-Roberts
frequency,4949.0,NaN,NaN,2741.0,NaN,1046.0,NaN,NaN,NaN,NaN,...,666.0,126.0,NaN,NaN,NaN,2571.0,937.0,NaN,NaN,NaN


In [5]:
traffic = {}

# Create a dictionary based off the item name and the quantity of that item
for index, row in chip.iterrows():
    # We must include the second condition since the first time we add a neighborhood, it might be NaN, which we cannot use
    if row["neighborhood"] not in traffic and row["average_daily_car_traffic"] != "":
        traffic[row["neighborhood"]] = row["average_daily_car_traffic"]
        
print(traffic)

{'Polish Hill': 4949.0, 'Central Lawrenceville': nan, 'Windgap': nan, 'Squirrel Hill South': 2741.0, 'Perry North': nan, 'Central Northside': 1046.0, 'Squirrel Hill North': nan, 'Central Oakland': nan, 'East Liberty': nan, 'Greenfield': nan, 'Bluff': nan, 'Crafton Heights': 11500.0, 'Shadyside': 6793.0, 'Upper Lawrenceville': nan, 'Highland Park': 7688.0, 'North Shore': 10350.0, 'Mount Washington': 3860.0, 'Brookline': 9709.0, 'South Shore': nan, 'South Side Flats': nan, 'Bloomfield': 1120.0, 'Larimer': 8512.0, 'Friendship': 4448.0, 'South Oakland': nan, 'Point Breeze': 12335.0, 'Westwood': nan, 'Regent Square': 16729.0, 'Knoxville': 527.0, 'Strip District': 6443.0, 'West Oakland': nan, 'Beechview': 2046.0, 'South Side Slopes': nan, 'Upper Hill': 1530.0, 'North Oakland': nan, 'Stanton Heights': 4016.0, 'Manchester': 9405.0, 'Morningside': nan, 'Hazelwood': nan, 'Lower Lawrenceville': nan, 'Overbrook': nan, 'East Hills': 13788.0, 'Duquesne Heights': 5831.0, 'Lincoln-Lemington-Belmar': n

In [6]:
lowest_neighborhood = ""
lowest_neighborhood_average_traffic = 10000000 # starter number
# Check each number of average traffic to see if it is less than the current lowest value
for index, row in chip.iterrows():
    # We must include the second condition since the first time we add a neighborhood, it might be NaN, which we cannot use
    if row["average_daily_car_traffic"] < lowest_neighborhood_average_traffic:
        lowest_neighborhood_average_traffic = row["average_daily_car_traffic"]
        lowest_neighborhood = row["neighborhood"]
        
print("The neighorhood with the least amount of traffic is: " + lowest_neighborhood)
print("With a average daily traffic count of: " + str(lowest_neighborhood_average_traffic))

The neighorhood with the least amount of traffic is: Central Northside
With a average daily traffic count of: 23.0


## With this information, I wanted to see if Central Northside did indeed have the least amount of daily traffic. I looked at the spreadsheet and it turned out this was only one of the datapoints. Another Central Northside datapoint had 3190 as the average_daily_car_traffic. Now knowing that this code is faulty, I decided that I would need to calculate an average for all of neighborhoods that have multiple datapoints.

In [26]:
import statistics as st
traffic_count = []
temp_array = []
neighborhoods = []
# take each traffic count data point for all neighborhoods, calculate an average, and then create a dataframe from it

# Start by creating an list of all the neighborhoods
for index, row in chip.iterrows():
    if row["neighborhood"] not in neighborhoods:
        neighborhoods.append(row["neighborhood"])
# Now for each neighborhood, add each datapoint it has to a list, calculate the average, then append that list to traffic_count
for neighborhood in neighborhoods:
    for index, row in chip.iterrows():
        if row["neighborhood"] == neighborhood and row["average_daily_car_traffic"] != "nan":
                temp_array.append(row["average_daily_car_traffic"])
    
    # After we add the datapoints for the neighborhood, we calculate the average and then add it to traffic count
    if len(temp_array) > 0:
        traffic_average = st.mean(temp_array)
    traffic_count.append(traffic_average)
    # Now we clear temp_array for use in the next iteration of the loop
    temp_array.clear()

# Print the final product
print(str(traffic_count))

[2649.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11500.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 2443.5, nan, nan, nan, 16729.0, 3823.0, nan, nan, nan, nan, nan, nan, 4068.0, 2129.3, nan, nan, nan, nan, nan, 5831.0, nan, nan, nan, nan, nan, nan, nan, 2386.0, 1924.0, nan, nan, 477.0, 2217.1428571428573, nan, nan, nan, 4964.0, nan, nan, nan, 1226.0, nan, 419.0, 666.0, 126.0, 126.0, nan, nan, 2571.0, nan, nan, nan, nan]


## I wanted to make sure that this data was correct, so I manually caclulated the average for the first neighborhood, which is Polish hill. I got the number that is first in the array for the previous code cell, so I know my work is correct.

In [13]:
import statistics as st
traffic_count = []
temp_array = []
neighborhoods = []
# take each traffic count data point for all neighborhoods, calculate an average, and then create a dataframe from it

for index, row in chip.iterrows():
    if row["neighborhood"] == "Polish Hill":
        temp_array.append(row["average_daily_car_traffic"])
    
# After we add the datapoints for the neighborhood, we calculate the average and then add it to traffic count
if len(temp_array) > 0:
    traffic_average = st.mean(temp_array)
    traffic_count.append(traffic_average)
temp_array.clear()
print(str(traffic_count))

[2649.0]


## My new problem is that I cannot get rid of nan, so I must create a query for the average_daily_car_traffic and create a new dataframe from those datapoints to only include the datapoints that have numbers.
## I then create a dictionary from the lists of neighborhoods and average daily car traffic to then display

In [9]:
import statistics as st
# We cut out the rows in which the average_daily_car_traffic is nan
chip.dropna(subset=["average_daily_car_traffic"],inplace=True)
traffic_count = []
temp_array = []
neighborhoods = []
traffic_dictionary = {}
# take each traffic count data point for all neighborhoods, calculate an average, and then create a dataframe from it

# Start by creating an list of all the neighborhoods
for index, row in chip.iterrows():
    if row["neighborhood"] not in neighborhoods:
        neighborhoods.append(row["neighborhood"])
# Now for each neighborhood, add each datapoint it has to a list, calculate the average, then append that list to traffic_count
for neighborhood in neighborhoods:
    for index, row in chip.iterrows():
        if row["neighborhood"] == neighborhood and row["average_daily_car_traffic"] != "nan":
                temp_array.append(row["average_daily_car_traffic"])
    
    # After we add the datapoints for the neighborhood, we calculate the average and then add it to traffic count
    if len(temp_array) > 0:
        traffic_average = st.mean(temp_array)
    traffic_count.append(traffic_average)
    # Now we clear temp_array for use in the next iteration of the loop
    temp_array.clear()

# Print the final product
traffic_dictionary = {neighborhoods[i]: traffic_count[i] for i in range(len(neighborhoods))}
print(traffic_dictionary)
print(len(traffic_dictionary))

{'Polish Hill': 2649.0, 'Squirrel Hill South': 3050.076923076923, 'Central Northside': 1189.0, 'Bluff': 1788.3333333333333, 'Crafton Heights': 11500.0, 'Shadyside': 4408.8, 'Highland Park': 1564.7333333333333, 'North Shore': 10350.0, 'East Liberty': 4658.818181818182, 'Mount Washington': 5259.555555555556, 'Brookline': 3914.4, 'Squirrel Hill North': 3939.1111111111113, 'Bloomfield': 3078.6428571428573, 'Larimer': 3771.3, 'Friendship': 2443.5, 'Point Breeze': 3905.25, 'Regent Square': 16729.0, 'Central Lawrenceville': 2249.8333333333335, 'Knoxville': 3823.0, 'Central Oakland': 3768.6666666666665, 'Strip District': 9691.666666666666, 'Greenfield': 2386.0, 'Windgap': 1531.0, 'Beechview': 1418.3333333333333, 'Upper Hill': 953.3333333333334, 'Stanton Heights': 4068.0, 'Manchester': 2129.3, 'South Side Slopes': 3778.1428571428573, 'Perry North': 2123.777777777778, 'North Oakland': 5955.25, 'East Hills': 8126.5, 'Duquesne Heights': 5831.0, 'Sheraden': 2844.0, 'Morningside': 2049.285714285714,

# Conclusion

## Considering the fact that higher traffic and higher speeds increase the likelyhood of accident and severity of injury, the columns of this data set we are taking into consideration are average_daily_car_traffic and percent_over_limit.

## 1. Create a dictionary of neighborhoods and percent_over_limit 
## 2. Check which neighborhoods have the least amount of traffic
## 3. Check which neighborhoods have the least percent_over_limit